### This file contains a preview of all the data pulled for S&P500 companies

![ratios definition](Ratios_def.png)

In [1]:
import datetime
import pandas as pd
import wrds

# Connect to WRDS
conn = wrds.Connection()

# Date
start_date = datetime.date(2020, 1, 1)
end_date = datetime.date(2020, 12, 31)

#print(conn.list_libraries())

WRDS recommends setting up a .pgpass file.
Created .pgpass file successfully.
Loading library list...
Done


In [18]:
# Open credit ratings file
creditRatingsFile = "data/credit_ratings_raw.csv"

creditRatingsDf = pd.read_csv(creditRatingsFile)
# creditRatingsDf = creditRatingsDf[creditRatingsDf['tic'].isin(tickers)]
creditRatingsDf['datadate'] = pd.to_datetime(creditRatingsDf['datadate'])

print(creditRatingsDf['datadate'].max())

# get the latest credit rating
latestCreditRatingsDf = creditRatingsDf[creditRatingsDf['datadate'] == creditRatingsDf['datadate'].max()]

# rename splticrm to long term credit rating
latestCreditRatingsDf = latestCreditRatingsDf.rename(columns={'splticrm': 'long_term_credit_rating'})

latestCreditRatingsDf = latestCreditRatingsDf[['tic', 'long_term_credit_rating']]

# count number of unique tic values
print(latestCreditRatingsDf['tic'].nunique())

# save unique tic values to list
creditRatingsTickers = latestCreditRatingsDf['tic'].tolist()

2017-02-28 00:00:00
1830


In [5]:
# get a list of tickers
tickers = creditRatingsTickers

tickerString = []
for ticker in tickers:
    tickerString.append("ticker = '" + ticker + "'")

tickerString = ' or '.join(tickerString)

# get the data
firmRatioDf = conn.raw_sql("select * from wrdsapps_finratio.firm_ratio where " + tickerString )

In [6]:
# Save to csv
saveToFile = True
if(saveToFile):
    firmRatioDf.to_csv('data/firm_ratio_russell.csv')
else:
    firmRatioDf = pd.read_csv('data/firm_ratio_russell.csv')


In [7]:
# treat public_date as date
firmRatioDf['public_date'] = pd.to_datetime(firmRatioDf['public_date'])

firmRatioDf['public_date'].value_counts()


2021-12-31    853
2021-11-30    851
2021-10-31    839
2021-09-30    839
2021-08-31    837
             ... 
1970-03-31    218
1970-02-28    218
1970-09-30    218
1970-08-31    217
1970-01-31    217
Name: public_date, Length: 624, dtype: int64

In [8]:
# treat public_date as date
firmRatioDf['public_date'] = pd.to_datetime(firmRatioDf['public_date'])

latestOnly = firmRatioDf[firmRatioDf['public_date'] == datetime.datetime(2017, 2, 28)]

columnsOfInterest = ['public_date', 'ticker', 'roa', 'intcov_ratio', 'quick_ratio', 'intcov', 'bm', 'npm', 'debt_assets', 'debt_ebitda', 'inv_turn', 'cash_ratio']

ratiosOfInterestDf = latestOnly[columnsOfInterest]

ratiosOfInterestDf

,public_date,ticker,roa,intcov_ratio,quick_ratio,intcov,bm,npm,debt_assets,debt_ebitda,inv_turn,cash_ratio
37,2017-02-28,AAL,0.157149,5.798263,0.665369,3.583012,0.160266,0.066600,0.926181,3.107877,26.259581,0.504397
535,2017-02-28,PNW,0.086478,4.160918,0.417366,3.148717,0.916132,0.126343,0.691587,3.222494,7.668622,0.006869
1493,2017-02-28,AMD,-0.018351,-0.833333,1.321694,-2.185897,0.040272,-0.116339,0.874737,-24.322034,4.006998,0.939079
2109,2017-02-28,APD,0.140038,10.662610,0.849159,8.235970,0.250545,0.142316,0.577241,2.274587,6.610056,0.217685
2833,2017-02-28,ABT,0.105949,17.004695,3.654955,5.990610,0.369329,0.050976,0.606634,4.423317,3.222730,2.819069
...,...,...,...,...,...,...,...,...,...,...,...,...
284574,2017-02-28,GWRE,0.023065,NaN,6.239675,NaN,0.188061,0.020051,0.131634,0.000000,NaN,5.520838
284652,2017-02-28,ACHC,0.112626,2.454940,1.234579,1.033878,0.782834,0.002185,0.637249,5.666243,411.877389,0.166430
284786,2017-02-28,TRIP,0.122767,13.916667,2.245863,11.000000,0.226564,0.081081,0.328865,0.638060,NaN,1.725768
284840,2017-02-28,VAC,0.112692,7.292518,NaN,5.028155,0.461546,0.075831,0.620385,2.733527,1.568138,NaN


In [9]:
# save to csv
ratiosOfInterestDf.to_csv('ratios_of_interest_russell1000.csv')

In [14]:
# Merge credit ratings with ratios
mergedDf = pd.merge(ratiosOfInterestDf, latestCreditRatingsDf, left_on='ticker', right_on='tic', how='inner')

mergedDf.drop(columns=['tic'], inplace=True)

mergedDf

,public_date,ticker,roa,intcov_ratio,quick_ratio,intcov,bm,npm,debt_assets,debt_ebitda,inv_turn,cash_ratio,long_term_credit_rating
0,2017-02-28,AAL,0.157149,5.798263,0.665369,3.583012,0.160266,0.066600,0.926181,3.107877,26.259581,0.504397,BB-
1,2017-02-28,PNW,0.086478,4.160918,0.417366,3.148717,0.916132,0.126343,0.691587,3.222494,7.668622,0.006869,A-
2,2017-02-28,AMD,-0.018351,-0.833333,1.321694,-2.185897,0.040272,-0.116339,0.874737,-24.322034,4.006998,0.939079,CCC+
3,2017-02-28,APD,0.140038,10.662610,0.849159,8.235970,0.250545,0.142316,0.577241,2.274587,6.610056,0.217685,A
4,2017-02-28,ABT,0.105949,17.004695,3.654955,5.990610,0.369329,0.050976,0.606634,4.423317,3.222730,2.819069,BBB
...,...,...,...,...,...,...,...,...,...,...,...,...,...
485,2017-02-28,MPC,0.102459,3.966454,0.664008,2.875399,0.655186,0.021027,0.522595,2.357716,9.143277,0.124825,BBB
486,2017-02-28,FBHS,0.155610,13.721757,1.084599,9.627615,0.307874,0.082730,0.539241,1.838043,5.663753,0.298410,BBB+
487,2017-02-28,XYL,0.118947,8.964912,1.063813,5.561404,0.286189,0.068947,0.659098,3.577039,4.521466,0.248788,BBB
488,2017-02-28,ACHC,0.112626,2.454940,1.234579,1.033878,0.782834,0.002185,0.637249,5.666243,411.877389,0.166430,B+


In [15]:
# save to csv
mergedDf.to_csv('ratios_with_credit_ratings_russell1000.csv')